In [50]:
#load "../include/Data.fsx"

In [51]:
open System
open System.Collections.Generic
open System.Linq;

open FSharp.Interop.Dynamic

open Sylvester.Data

//Download a schema from a CSV file 
let msft = new CsvFile("https://raw.githubusercontent.com/matplotlib/sample_data/master/msft.csv")

// Set the first CSV field to a DateTime
msft.[0].Type <- typeof<DateTime>

// Set the remaining fields to floating point
for j in 1..msft.Fields.Count - 1 do msft.[j].Type <- typeof<float> 

// Show all the field labels in the schema
msft.Select(fun f -> f.Label + ":" + f.Type.Name)

seq ["Date:DateTime"; "Open:Double"; "High:Double"; "Low:Double"; ...]

In [52]:
//Now create a frame from the fields defined
let df = new Frame(msft)
df

seq
  [seq [92433800.0; 29.79]; seq [67268096.0; 29.34]; seq [47221600.0; 28.34];
   seq [52060600.0; 28.74]; ...]

In [53]:
// The Date property is a dynamic member of df with a static series type
let date:Cd = df?Date
date

seq
  [09/19/2003 00:00:00; 09/18/2003 00:00:00; 09/17/2003 00:00:00;
   09/16/2003 00:00:00; ...]

The High property is a series of floating-point data.

In [54]:
for i in df?High do printf "%.2f " i

29.97 29.51 28.95 28.95 28.61 28.40 28.11 28.18 28.71 28.92 28.75 28.47 28.40 27.30 26.55 26.58 26.58 26.67 26.54 26.95 26.73 26.53 26.65 25.83 25.66 25.71 25.89 25.77 25.99 25.98 25.81 26.19 26.54 26.41 26.51 26.99 26.57 26.90 27.00 26.95 26.92 26.65 26.56 26.91 27.23 27.27 27.62 27.53 27.81 27.45 27.42 27.70 27.80 27.55 26.95 26.93 26.20 26.12 26.34 26.51 25.99 26.04 26.24 26.38 26.39 

In [55]:
query {
    for r in df do
    sortByDescending r?Volume
    select r.["Date"]
}

seq
  [09/03/2003 00:00:00; 07/02/2003 00:00:00; 09/19/2003 00:00:00;
   07/07/2003 00:00:00; ...]

In [56]:
// Select a tuple of 2 fields from the frame
query {
    for r in df do 
    sortBy r?High 
    take 10
    select {|Date=r?Date; High=r?High|}
} |> Util.Table

Date,High
08/15/2003 00:00:00,25.66
08/14/2003 00:00:00,25.71
08/12/2003 00:00:00,25.77
08/07/2003 00:00:00,25.81
08/18/2003 00:00:00,25.83
08/13/2003 00:00:00,25.89
08/08/2003 00:00:00,25.98
08/11/2003 00:00:00,25.99
06/25/2003 00:00:00,25.99
06/24/2003 00:00:00,26.04


In [57]:
// The original MSFT dataset has 7 series
df.Columns.Count

7

In [58]:
query {for r in df do select (r.["Date"])}

seq
  [09/19/2003 00:00:00; 09/18/2003 00:00:00; 09/17/2003 00:00:00;
   09/16/2003 00:00:00; ...]

Rows in data frames forward data access calls to their parent frame. No additional storage for querying by row or column is allocated.

In [59]:
printfn "%.4f" df.[16]?High

26.5800


### Examples

Sylvester can make exploratory data analysis with F# easier and faster than existing .NET libraries. Using LINQ can make queries less verbose and easier to understand than other .NET data frame libraries like [Deedle](https://bluemountaincapital.github.io/Deedle/).

In [63]:
let titanic = new CsvFile("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
titanic.["Pclass"].Type <- typeof<int>
let dt = new Frame(titanic)

query {
    for r in dt do
    groupBy r?Pclass into g
    sortBy g.Key
    select (
        let survived = (g.Where(fun p -> p?Survived = "1").Count()) |> float
        let died = (g.Where(fun p -> p?Survived = "0").Count()) |> float
        let ctotal = survived + died
        let psurvived = round(100.0 * survived / ctotal)
        let pdied = round(100.0 * died / ctotal) 
        {|Class=g.Key; Died=pdied; Survived=psurvived|}
)} |> Util.Table

Class,Died,Survived
1,37,63
2,53,47
3,76,24
